In [ ]:
# Primer paso, instalación de las herramientas que voy a necesitar a lo largo del análisis Exploratorio
# de datos (EDA).

In [1]:
pip install pandas matplotlib seaborn openpyxl


   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 8.1/8.1 MB 58.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 70.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 2.6/2.6 MB 62.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
